In [1]:
%cd ../..
%load_ext autoreload
%autoreload 2

/home/jupyter/map-ephys


In [2]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
from pipeline import lab, get_schema_name, experiment, foraging_model, ephys, foraging_analysis, histology, ccf, report, psth_foraging
from pipeline.plot import behavior_plot, unit_characteristic_plot, unit_psth, histology_plot, PhotostimError, foraging_plot
from pipeline.plot.foraging_model_plot import plot_session_model_comparison, plot_session_fitted_choice, plot_unit_all_in_one

foraging_sessions = (foraging_analysis.SessionTaskProtocol & 'session_task_protocol=100').proj()
import datajoint as dj; dj.conn().connect()

[2022-11-19 06:44:50,199][INFO]: Connecting map-superuser@datajoint.mesoscale-activity-map.org:3306
[2022-11-19 06:44:50,255][INFO]: Connected map-superuser@datajoint.mesoscale-activity-map.org:3306


In [27]:
cache_folder = '~/s3-drive/report/st_cache/'

def fetch_ephys_units():
    # df = pd.DataFrame(all_unit_qc.fetch())
    from pipeline import experiment, ephys, lab, psth_foraging, report, foraging_analysis
    
    t_iti_delta_Q = (psth_foraging.UnitPeriodLinearFit.Param & 'period in ("iti_all")' & 'multi_linear_model = "Q_rel + Q_tot + rpe"' 
    & 'var_name = "relative_action_value_ic"').proj(iti_dQ='t', iti_dQ_abs='ABS(t)', _='period')
    
    t_trial_delta_Q = (psth_foraging.UnitPeriodLinearFit.Param & 'period in ("go_to_end")' & 'multi_linear_model = "Q_rel + Q_tot + rpe"' 
    & 'var_name = "relative_action_value_ic"').proj(trial_dQ='t', trial_dQ_abs='ABS(t)', __='period')
    
    df_all_unit = pd.DataFrame((
                report.UnitLevelForagingEphysReportAllInOne.key_source
                * ephys.Unit.proj('unit_uid', 'unit_amp', 'unit_snr')
                * lab.WaterRestriction.proj(h2o='water_restriction_number')
                * experiment.Session.proj(session_date="cast(concat(session_date) as datetime)")
                # * ephys.ClusterMetric.proj('presence_ratio', 'amplitude_cutoff')
                # * ephys.UnitStat
                * t_iti_delta_Q * t_trial_delta_Q
                ).fetch())
    
    df_all_unit.drop(['clustering_method', 'behavior_model', 'multi_linear_model', 'var_name', '_', '__'], axis=1, inplace=True)
    df_all_unit.rename(columns={'insertion_number': 'ins'}, inplace=True)
    
    df_all_unit.to_pickle(cache_folder + 'ephys_units.pkl')
    
    return df_all_unit

df_all_unit = fetch_ephys_units()
df_all_unit

,subject_id,session,ins,unit,unit_uid,unit_amp,unit_snr,h2o,session_date,iti_dQ,iti_dQ_abs,trial_dQ,trial_dQ_abs
0,473360,49,2,13,13,77.286874,3.128028,HH08,2021-08-12,0.147769,0.147769,2.520250,2.520252
1,473360,49,2,16,16,224.812200,8.996118,HH08,2021-08-12,1.381970,1.381965,0.789881,0.789881
2,473360,49,2,24,24,78.242861,4.015200,HH08,2021-08-12,-0.984233,0.984233,-1.275100,1.275098
3,473360,49,2,25,25,84.003283,3.989731,HH08,2021-08-12,-1.965720,1.965724,-0.743375,0.743375
4,473360,49,2,27,27,93.121246,3.631522,HH08,2021-08-12,1.433060,1.433064,0.554317,0.554317
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4583,494691,18,3,72,72,73.834608,2.783918,HH16,2021-12-09,0.514781,0.514781,-1.971400,1.971395
4584,494691,18,3,80,80,191.754456,7.150332,HH16,2021-12-09,-0.639362,0.639362,0.053673,0.053673
4585,494691,18,3,90,90,98.785274,4.242584,HH16,2021-12-09,-5.790850,5.79085,-5.545650,5.545646
4586,494691,18,3,92,92,106.629797,4.463012,HH16,2021-12-09,-7.978690,7.978694,-8.058160,8.058163


In [7]:
def fetch_sessions():
    # with st.spinner(f'Connect to datajoint...'):
    from pipeline import experiment, ephys, lab, psth_foraging, report, foraging_analysis, histology
    import datajoint as dj; dj.conn().connect()
    
    # with st.spinner(f'Fetching...'):
    foraging_sessions = (foraging_analysis.SessionTaskProtocol & 'session_task_protocol=100').proj()
    insertion_numbers = foraging_sessions.aggr(foraging_sessions * ephys.ProbeInsertion, ..., 
                                                    #   keep_all_rows=True, ephys_insertions='IF(COUNT(insertion_number), "yes", "no")')
                                                keep_all_rows=True, ephys_ins='COUNT(insertion_number)')
    if_histology = foraging_sessions.aggr(foraging_sessions * histology.ElectrodeCCFPosition.ElectrodePosition, ...,
                                          keep_all_rows=True, histology='IF(COUNT(ccf_x)>0, "yes", "no")')
    if_photostim_from_behav = foraging_sessions.aggr(foraging_sessions * experiment.PhotostimForagingTrial, ...,
                                          keep_all_rows=True, photostim_behav='IF(COUNT(trial)>0, "yes", "no")')
    if_photostim_from_ephys = foraging_sessions.aggr(foraging_sessions * (ephys.TrialEvent & 'trial_event_type LIKE "laser%"'), ...,
                                          keep_all_rows=True, photostim_NI='IF(COUNT(trial)>0, "yes", "no")')

    df_sessions = pd.DataFrame(((experiment.Session & foraging_sessions)
                                * lab.WaterRestriction.proj(h2o='water_restriction_number')
                                * insertion_numbers
                                * if_histology
                                * if_photostim_from_behav
                                * if_photostim_from_ephys)
                               .proj(..., '-rig', '-username', '-session_time')
                               .fetch()
                                )
    # df_sessions['session_date'] = pd.to_datetime(df_sessions['session_date'], format="%Y-%m-%d")
    df_sessions.to_pickle(cache_folder + 'sessions.pkl')
    
    return df_sessions

fetch_sessions()

,subject_id,session,session_date,h2o,ephys_ins,histology,photostim_behav,photostim_NI
0,447921,1,2019-09-11,HC16,0,no,no,no
1,447921,2,2019-09-12,HC16,0,no,no,no
2,447921,3,2019-09-13,HC16,0,no,no,no
3,447921,4,2019-09-14,HC16,0,no,no,no
4,447921,5,2019-09-15,HC16,0,no,no,no
...,...,...,...,...,...,...,...,...
1792,634898,15,2022-09-02,KH_FB39,0,no,no,no
1793,634898,16,2022-09-06,KH_FB39,0,no,no,no
1794,634898,17,2022-09-07,KH_FB39,0,no,no,no
1795,634898,18,2022-09-08,KH_FB39,0,no,no,no
